<a href="https://colab.research.google.com/github/kursatkara/MAE_5020_S24/blob/master/08_03_nn_pytorch_lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NN with Pytorch Lightning**

Creating a basic three-layer neural network for the MNIST dataset with PyTorch Lightning, including training and validation loops.

Regenerated from https://www.geeksforgeeks.org/pytorch-vs-pytorch-lightning/

##Install

In [ ]:
!pip install pytorch_lightning -qq

##Import
* PyTorch
* PyTorch Lightning
* MNIST dataset: The dataset contains 70,000 images of handwritten digits from 0 to 9. Each image is a 28x28 pixel grayscale representation of a digit. MNIST is commonly used as a benchmark for evaluating the performance of algorithms on image recognition tasks. It's particularly popular in introductory machine learning and computer vision courses because the dataset is small enough to be easily handled on a standard computer, yet large enough to require and demonstrate the effectiveness of the neural networks or other machine learning techniques applied to it.
* Adam optimizer: Adam (Adaptive Moment Estimation) optimizer is the extended version of stochastic gradient descent which could be implemented in various deep learning applications

In [ ]:
import pytorch_lightning as pl

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.optim import Adam

##NN Model
MyModel class defines the neural network architecture, the forward pass, the training step, the validation step, and the configuration of the optimizer.

The model consists of three fully connected layers with 256, 128, and 10 neurons, and a final softmax layer for outputting probabilities. The output layer has 10 neurons, corresponding to the 10 classes of digits in the MNIST dataset. The input is passed through each layer and transformed using the ReLU activation function.

The forward method takes in an input tensor x, reshapes it to have the correct number of dimensions, and passes it through the neural network using the self.model module.

In [ ]:
class MyModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Reshape the input
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.CrossEntropyLoss()(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.CrossEntropyLoss()(y_hat, y)
        self.log('val_loss', loss)
        # Calculate accuracy
        correct = (y_hat.argmax(1) == y).sum().item()
        total = y.size(0)
        self.log('accuracy', correct / total, on_step=False, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.001)


##Loading the dataset

The MNIST dataset is loaded using MNIST class from torchvision.datasets. The training set and validation set are split into separate DataLoader objects for training and validation.

In [ ]:
# Load the MNIST dataset
train_dataset = MNIST(root='.', train=True, transform=ToTensor(), download=True)
val_dataset = MNIST(root='.', train=False, transform=ToTensor())

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)


##The NN Model

In [ ]:
# Initialize the model
model = MyModel()


In [ ]:
# Initialize the trainer
trainer = pl.Trainer(max_epochs=10, accelerator = "gpu" if torch.cuda.is_available() else "cpu")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
# Train the model
trainer.fit(model, train_loader, val_loader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 235 K 
-------------------------------------
235 K     Trainable params
0         Non-trainable params
235 K     Total params
0.941     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
#Validate the model
trainer.validate(model, val_loader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.9747999906539917     │
│         val_loss          │    1.4867440462112427     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 1.4867440462112427, 'accuracy': 0.9747999906539917}]